<a href="https://colab.research.google.com/github/Naldershof/Wikigraphs/blob/main/WikiLinksStarting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
try:  
  import wikipedia
except:
  !pip install wikipedia
  import wikipedia

try:
  import umap
  import umap.plot
except:
  !pip install umap-learn
  !pip install umap-learn[plot]
  import umap
  import umap.plot

try:
  import plotly.express as px
except:
  !pip install plotly
  import plotly.express as px

import seaborn as sns
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# TODO TOMORROW
# Filter by pageviews instead
# Can get page views from sample GCP project, should be good enough even if the rest of this isn't super fast. 

In [4]:
#https://wikipedia.readthedocs.io/en/latest/code.html#api
Philosophy = wikipedia.page("Philosophy")

In [39]:
%%time
# 1000 may be too many, 100 takes like 40 seconds, so should take around 5 min 
# Fuck it, let's go crazy, 5000, thirty minutes or whatever, I'll go ddo something else for a bit
max_links = min(5000, len(Philosophy.links))
print(max_links)

# So while we're dealing with getting those file uploads, 
# lets just try selecting a random 100 of the links from philosophy
# getting the links between them, and then creating a graph off that 
# will let us start getting things moving

# Yeah this is not very performant... We're probably gonna have to just
# use the actual stuff.   
link_dict = {}
all_links = []

random_links = np.random.choice(Philosophy.links, max_links, replace=False)

# Record things that we don't have and remove them from all our lists
missing_pages = []

for link in random_links:
  try:
    page = wikipedia.page(link)
    page_links = page.links
    # Thissss is going to get real nasty when I remove the intersection shit, GET READY FOR SOME NONSENSE 
    #link_dict[link] = list(set(random_links).intersection(page_links))
    # Yeah I should do the direction 2 thing but that's a fuckload of data... I might have to just give up and actually use the real data...
    link_dict[link] = page_links
    all_links.extend(page_links)
  except:
    missing_pages.append(link)

1245


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.




CPU times: user 51.4 s, sys: 2.92 s, total: 54.3 s
Wall time: 4min 33s


In [64]:
# And next, come back and get links from all the new pages to pages in the original dataset, should basically make it so that we have 
# links of degree 2 from the original page, could be better, but not bad at all. 
# Nah, that's too many, but what if instead we trim this down to only links which are seen at least twice? five times? Lets see how many. 
df = pd.DataFrame.from_dict(Counter(all_links), orient='index').reset_index()
df.columns = ['title', 'counts']
print("Limiting to above {}".format(df.counts.quantile(0.975)))
lim_df = df[df.counts >= df.counts.quantile(0.975)]
print(lim_df.shape)

Limiting to above 28.0
(3590, 2)


In [70]:
new_pages = list(lim_df['title'])
actually_new_pages = [x for x in new_pages if x not in random_links]
len(actually_new_pages)

2606

In [71]:
%%time
for link in actually_new_pages:
  try:
    page = wikipedia.page(link)
    page_links = page.links
    # Thissss is going to get real nasty when I remove the intersection shit, GET READY FOR SOME NONSENSE 
    #link_dict[link] = list(set(random_links).intersection(page_links))
    # Yeah I should do the direction 2 thing but that's a fuckload of data... I might have to just give up and actually use the real data...
    link_dict[link] = page_links
    all_links.extend(page_links)
  except:
    #missing_pages.append(link)
    pass

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.




In [74]:
for missing in missing_pages:
  # Yeah, this is nonsense, fix it, I think instead just prune isolates should take care of it
  # This code be way better but I'm lazy rn
  try:
    del(link_dict[missing])
  except:
    pass

In [75]:
G = nx.from_dict_of_lists(link_dict, nx.DiGraph)
#G = nx.from_dict_of_lists(link_dict, nx.Graph)
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
# Ok so remove nodes that are not in the union of actually_new_pages and random_links
nodes_of_importance = list(random_links + actually_new_pages)

In [76]:
adjacency = nx.linalg.graphmatrix.adjacency_matrix(G)
print(adjacency.shape)

(293120, 293120)


In [ ]:
%%time
mapper = umap.UMAP(metric='cosine', random_state=42, 
                   low_memory=False, n_neighbors=10).fit(adjacency)

/usr/local/lib/python3.7/dist-packages/umap/umap_.py:126: UserWarning:

A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 848 edges.
It has only fully disconnected 64 vertices.
Use umap.utils.disconnected_vertices() to identify them.



In [ ]:
umap.plot.points(mapper, values=np.arange(adjacency.shape[0]), theme='viridis')

In [ ]:
proj_2d = mapper.fit_transform(adjacency)

In [ ]:
fig_2d = px.scatter(
    proj_2d, x=0, y=1, text=list(nx.nodes(G))    
)

fig_2d.show()

In [ ]:
# Will need to get the pageviews for some of this so I can limit the
# links and make some nice little things
# Example here of getting this into google clcoud
# https://codelabs.developers.google.com/codelabs/cloud-bigquery-wikipedia#4

# Ehh, just save to drive, these are pretty small (like 500mb each)
# https://dumps.wikimedia.org/other/pageview_complete/readme.html
# All 8/2021 is only 2.5gb, not the end of the world
# https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/

In [ ]:
#Basically I want to play with knowledge graphs cause they're fun